In [1]:
from pyspark.sql import functions as F
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("OULAD Analysis") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()


25/02/11 22:04:45 WARN Utils: Your hostname, mickael resolves to a loopback address: 127.0.1.1; using 192.168.1.253 instead (on interface wlp2s0)
25/02/11 22:04:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/11 22:04:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
data_courses = spark.read.csv("file:///home/mickael/data_engineer/hadoop/oulad/courses.csv", header=True, inferSchema=True)
data_assessments = spark.read.csv("file:///home/mickael/data_engineer/hadoop/oulad/assessments.csv", header=True, inferSchema=True)
data_studentAssessment = spark.read.csv("file:///home/mickael/data_engineer/hadoop/oulad/studentAssessment.csv", header=True, inferSchema=True)
data_studentInfo = spark.read.csv("file:///home/mickael/data_engineer/hadoop/oulad/studentInfo.csv", header=True, inferSchema=True)
data_studentRegistration = spark.read.csv("file:///home/mickael/data_engineer/hadoop/oulad/studentRegistration.csv", header=True, inferSchema=True)

In [12]:
data_courses.show(3)
data_assessments.show(15)

+-----------+-----------------+--------------------------+
|code_module|code_presentation|module_presentation_length|
+-----------+-----------------+--------------------------+
|        AAA|            2013J|                       268|
|        AAA|            2014J|                       269|
|        BBB|            2013J|                       268|
+-----------+-----------------+--------------------------+
only showing top 3 rows

+-----------+-----------------+-------------+---------------+----+------+
|code_module|code_presentation|id_assessment|assessment_type|date|weight|
+-----------+-----------------+-------------+---------------+----+------+
|        AAA|            2013J|         1752|            TMA|  19|  10.0|
|        AAA|            2013J|         1753|            TMA|  54|  20.0|
|        AAA|            2013J|         1754|            TMA| 117|  20.0|
|        AAA|            2013J|         1755|            TMA| 166|  20.0|
|        AAA|            2013J|         1756

In [67]:
data_studentAssessment.show(5)
data_studentInfo.show(5)

+-------------+----------+--------------+---------+-----+
|id_assessment|id_student|date_submitted|is_banked|score|
+-------------+----------+--------------+---------+-----+
|         1752|     11391|            18|        0|   78|
|         1752|     28400|            22|        0|   70|
|         1752|     31604|            17|        0|   72|
|         1752|     32885|            26|        0|   69|
|         1752|     38053|            19|        0|   79|
+-------------+----------+--------------+---------+-----+
only showing top 5 rows

+-----------+-----------------+----------+------+--------------------+--------------------+--------+--------+--------------------+---------------+----------+------------+
|code_module|code_presentation|id_student|gender|              region|   highest_education|imd_band|age_band|num_of_prev_attempts|studied_credits|disability|final_result|
+-----------+-----------------+----------+------+--------------------+--------------------+--------+--------+--

In [68]:
data_studentRegistration.show(5)


+-----------+-----------------+----------+-----------------+-------------------+
|code_module|code_presentation|id_student|date_registration|date_unregistration|
+-----------+-----------------+----------+-----------------+-------------------+
|        AAA|            2013J|     11391|             -159|               NULL|
|        AAA|            2013J|     28400|              -53|               NULL|
|        AAA|            2013J|     30268|              -92|                 12|
|        AAA|            2013J|     31604|              -52|               NULL|
|        AAA|            2013J|     32885|             -176|               NULL|
+-----------+-----------------+----------+-----------------+-------------------+
only showing top 5 rows



## **Basic** SQL Questions

### 2. Find the number of students enrolled in each course.


In [19]:
# - Find the number of students enrolled in each course.
data_studentInfo.createOrReplaceTempView("studentInfo")

spark.sql("select code_module, code_presentation, count(id_student) as count from studentInfo group by code_module, code_presentation").show(5)

+-----------+-----------------+-----+
|code_module|code_presentation|count|
+-----------+-----------------+-----+
|        FFF|            2013J| 2283|
|        BBB|            2013J| 2237|
|        BBB|            2014J| 2292|
|        EEE|            2014J| 1188|
|        BBB|            2014B| 1613|
+-----------+-----------------+-----+
only showing top 5 rows



In [18]:
data_studentInfo.select("code_module", "code_presentation").groupBy("code_module", "code_presentation").count().show(5)


+-----------+-----------------+-----+
|code_module|code_presentation|count|
+-----------+-----------------+-----+
|        FFF|            2013J| 2283|
|        BBB|            2013J| 2237|
|        BBB|            2014J| 2292|
|        EEE|            2014J| 1188|
|        BBB|            2014B| 1613|
+-----------+-----------------+-----+
only showing top 5 rows



### 4.Find the distinct modules available.


In [30]:
data_courses.select("code_module").distinct().show()

+-----------+
|code_module|
+-----------+
|        CCC|
|        BBB|
|        DDD|
|        FFF|
|        EEE|
|        AAA|
|        GGG|
+-----------+



In [29]:
data_courses.createOrReplaceTempView("courses")
spark.sql("select code_module from courses group by code_module ").show()

+-----------+
|code_module|
+-----------+
|        CCC|
|        BBB|
|        DDD|
|        FFF|
|        EEE|
|        AAA|
|        GGG|
+-----------+



In [32]:
data_studentInfo.show(5)

+-----------+-----------------+----------+------+--------------------+--------------------+--------+--------+--------------------+---------------+----------+------------+
|code_module|code_presentation|id_student|gender|              region|   highest_education|imd_band|age_band|num_of_prev_attempts|studied_credits|disability|final_result|
+-----------+-----------------+----------+------+--------------------+--------------------+--------+--------+--------------------+---------------+----------+------------+
|        AAA|            2013J|     11391|     M| East Anglian Region|    HE Qualification| 90-100%|    55<=|                   0|            240|         N|        Pass|
|        AAA|            2013J|     28400|     F|            Scotland|    HE Qualification|  20-30%|   35-55|                   0|             60|         N|        Pass|
|        AAA|            2013J|     30268|     F|North Western Region|A Level or Equiva...|  30-40%|   35-55|                   0|             60

### 6. Get all students who have withdrawn from their course


In [34]:
spark.sql("select code_module, code_presentation, id_student from studentInfo where final_result = 'Withdrawn'").show(5)

+-----------+-----------------+----------+
|code_module|code_presentation|id_student|
+-----------+-----------------+----------+
|        AAA|            2013J|     30268|
|        AAA|            2013J|     65002|
|        AAA|            2013J|     94961|
|        AAA|            2013J|    106247|
|        AAA|            2013J|    129955|
+-----------+-----------------+----------+
only showing top 5 rows



In [39]:
data_studentInfo.select("code_module", "code_presentation", "id_student").filter(data_studentInfo.final_result == 'Withdrawn').show(5)

+-----------+-----------------+----------+
|code_module|code_presentation|id_student|
+-----------+-----------------+----------+
|        AAA|            2013J|     30268|
|        AAA|            2013J|     65002|
|        AAA|            2013J|     94961|
|        AAA|            2013J|    106247|
|        AAA|            2013J|    129955|
+-----------+-----------------+----------+
only showing top 5 rows



### 8. List students who registered late (after course start date).


In [44]:
data_studentRegistration.select("code_module", "code_presentation", "id_student").filter(data_studentRegistration.date_registration > 0).show(5)

+-----------+-----------------+----------+
|code_module|code_presentation|id_student|
+-----------+-----------------+----------+
|        AAA|            2013J|    106247|
|        AAA|            2013J|    236284|
|        AAA|            2013J|    341872|
|        AAA|            2013J|   1472925|
|        AAA|            2014J|    199897|
+-----------+-----------------+----------+
only showing top 5 rows



In [45]:
data_studentRegistration.createOrReplaceTempView("studentRegistration")
spark.sql("select code_module, code_presentation, id_student from studentRegistration where date_registration > 0 ").show(5)

+-----------+-----------------+----------+
|code_module|code_presentation|id_student|
+-----------+-----------------+----------+
|        AAA|            2013J|    106247|
|        AAA|            2013J|    236284|
|        AAA|            2013J|    341872|
|        AAA|            2013J|   1472925|
|        AAA|            2014J|    199897|
+-----------+-----------------+----------+
only showing top 5 rows



### 10. Get the number of assessment submissions per student.


In [55]:
data_studentAssessment.select("id_student").groupby("id_student").count().show(5)

+----------+-----+
|id_student|count|
+----------+-----+
|    180753|    5|
|    324084|    5|
|   2057803|    5|
|    486656|   11|
|    502604|   11|
+----------+-----+
only showing top 5 rows



In [56]:
data_studentAssessment.createOrReplaceTempView("studentAssessment")
spark.sql("select id_student, count(*) as count from studentAssessment group by id_student").show(5)

+----------+-----+
|id_student|count|
+----------+-----+
|    180753|    5|
|    324084|    5|
|   2057803|    5|
|    486656|   11|
|    502604|   11|
+----------+-----+
only showing top 5 rows



### 12. Find all assessments of module ‘AAA’.


In [57]:
data_assessments.show()

+-----------+-----------------+-------------+---------------+----+------+
|code_module|code_presentation|id_assessment|assessment_type|date|weight|
+-----------+-----------------+-------------+---------------+----+------+
|        AAA|            2013J|         1752|            TMA|  19|  10.0|
|        AAA|            2013J|         1753|            TMA|  54|  20.0|
|        AAA|            2013J|         1754|            TMA| 117|  20.0|
|        AAA|            2013J|         1755|            TMA| 166|  20.0|
|        AAA|            2013J|         1756|            TMA| 215|  30.0|
|        AAA|            2013J|         1757|           Exam|NULL| 100.0|
|        AAA|            2014J|         1758|            TMA|  19|  10.0|
|        AAA|            2014J|         1759|            TMA|  54|  20.0|
|        AAA|            2014J|         1760|            TMA| 117|  20.0|
|        AAA|            2014J|         1761|            TMA| 166|  20.0|
|        AAA|            2014J|       

In [58]:
data_assessments.createOrReplaceTempView("assessments")
spark.sql("select * from assessments where code_module = 'AAA'").show()

+-----------+-----------------+-------------+---------------+----+------+
|code_module|code_presentation|id_assessment|assessment_type|date|weight|
+-----------+-----------------+-------------+---------------+----+------+
|        AAA|            2013J|         1752|            TMA|  19|  10.0|
|        AAA|            2013J|         1753|            TMA|  54|  20.0|
|        AAA|            2013J|         1754|            TMA| 117|  20.0|
|        AAA|            2013J|         1755|            TMA| 166|  20.0|
|        AAA|            2013J|         1756|            TMA| 215|  30.0|
|        AAA|            2013J|         1757|           Exam|NULL| 100.0|
|        AAA|            2014J|         1758|            TMA|  19|  10.0|
|        AAA|            2014J|         1759|            TMA|  54|  20.0|
|        AAA|            2014J|         1760|            TMA| 117|  20.0|
|        AAA|            2014J|         1761|            TMA| 166|  20.0|
|        AAA|            2014J|       

## **Intermediate** SQL Questions



### 1. Get the average score per module.


In [5]:
data_studentAssessment.createOrReplaceTempView("studentAssessment")
data_assessments.createOrReplaceTempView("assessments")


spark.sql("select a.code_module, AVG(sa.score) as avg from assessments a join studentAssessment sa on a.id_assessment = sa.id_assessment group by a.code_module").show()

+-----------+-----------------+
|code_module|              avg|
+-----------+-----------------+
|        CCC| 73.2613978551429|
|        BBB| 76.7063682263431|
|        DDD|70.09079993509654|
|        FFF|77.70759006007047|
|        EEE|81.18006593963987|
|        AAA|69.03051493960585|
|        GGG|79.70049293460401|
+-----------+-----------------+



In [6]:
data_studentAssessment.join(data_assessments, data_studentAssessment.id_assessment == data_assessments.id_assessment).groupby("code_module").agg({"score": "avg"}).show()

+-----------+-----------------+
|code_module|       avg(score)|
+-----------+-----------------+
|        CCC| 73.2613978551429|
|        BBB| 76.7063682263431|
|        DDD|70.09079993509654|
|        FFF|77.70759006007047|
|        EEE|81.18006593963987|
|        AAA|69.03051493960585|
|        GGG|79.70049293460401|
+-----------+-----------------+



### 2.Find the students who scored the highest in any assessment.


In [10]:
max_scores = data_studentAssessment.groupby("id_assessment") \
    .agg(F.max("score").alias("max_score"))

max_scores.show(5)

max_scores.join(data_studentAssessment, 
    (data_studentAssessment.id_assessment == max_scores.id_assessment)
    & (data_studentAssessment.score == max_scores.max_score)
    ).show() 

+-------------+---------+
|id_assessment|max_score|
+-------------+---------+
|        15003|      100|
|        15004|      100|
|        34878|      100|
|        14997|       96|
|        34870|      100|
+-------------+---------+
only showing top 5 rows

+-------------+---------+-------------+----------+--------------+---------+-----+
|id_assessment|max_score|id_assessment|id_student|date_submitted|is_banked|score|
+-------------+---------+-------------+----------+--------------+---------+-----+
|         1752|       94|         1752|   2555340|            18|        0|   94|
|         1753|       95|         1753|    296332|            52|        0|   95|
|         1754|       95|         1754|   2458355|           117|        0|   95|
|         1755|       94|         1755|    376439|           166|        0|   94|
|         1755|       94|         1755|   2694424|           162|        0|   94|
|         1756|       98|         1756|   2536991|           215|        0|   98|
|  

### 3. Calculate the overall average score of students who passed.


In [21]:
data_studentAssessment.join(data_assessments, data_studentAssessment.id_assessment == data_assessments.id_assessment).filter((data_assessments.assessment_type == "Exam") & (data_studentAssessment.score > 50)).groupby(data_assessments.id_assessment).agg({"score": "avg"}).show()

+-------------+-----------------+
|id_assessment|       avg(score)|
+-------------+-----------------+
|        24299|78.56521739130434|
|        25340|67.14973262032086|
|        24290|79.94405594405595|
|        25354|72.13026315789473|
|        25361|75.16247139588101|
|        25368|71.98863636363636|
+-------------+-----------------+



### 4. Find courses where more than 50% of students passed.
### 5. Find students with a final result of ‘Distinction’ but never submitted an assessment.
### 6. Find assessments where more than 80% of students scored below 50.
### 7. Find students who submitted the same assessment multiple times.
### 8. Find courses with the highest dropout rate.
### 9. Calculate the number of interactions (clicks) per student.
### 10. Find the median score per module.
### 11. Calculate submission delay patterns.
### 12. Compare performance across age bands.